Below Function cleans the string given to it so that only Devanagari letters remain

In [1]:
def clean(contents):
    cleaned=""
    vowels = ['अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 'अं', 'अः', 'अँ']
    matra = ['ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ो', 'ै', 'ौ', 'ृ', 'ॄ', 'ॉ', 'ं','्','ँ']
    consonants = ['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ',
        'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
        'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
        'ष', 'स', 'ह', 'ळ', 'क्ष', 'ज्ञ']
    mapping= {'ा': 'आ', 'ि': 'इ', 'ी': 'ई', 'ु': 'उ', 'ू': 'ऊ', 'ृ': 'ऋ', 'ॄ': 'ॠ', 'ॅ': 'ऍ', 'े': 'ए', 'ै': 'ऐ', 'ो': 'ओ', 'ौ': 'औ', 'ं': 'अं', 'ः': 'अः', 'ँ': 'अँ'}
    for i in contents:
        if i in vowels or i in matra or i in consonants or i in mapping or i==' ':
            cleaned+=i
    return cleaned

In [3]:
import sentencepiece as spm
file=open('hi_100.txt', 'r')
contents = file.read()
contents=clean(contents)
file.close()
consonants=['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ',
              'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ',
              'ण', 'त', 'थ', 'द', 'ध', 'न', 'प',
              'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल',
              'व', 'श', 'ष', 'स', 'ह']
vowels=['अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ']
matras={'ा':'आ', 'ि':'इ', 'ी':'ई', 'ु':'उ', 'ू':'ऊ', 'े':'ए', 'ै':'ऐ', 'ो':'ओ', 'ौ':'औ', 'ं':'अं', 'ँ':'आँ'}

Below Function gives bigrams of a list of lists(Character and Syllables)

In [4]:
def bi_gram(Syllables):
    Bi=[]
    for i in range(len(Syllables)):
        for j in range(0,len(Syllables[i])-1):
            Bi.append(Syllables[i][j]+Syllables[i][j+1])
    return Bi

Below Function gives bigrams of a list of strings(Tokens)

In [5]:
def bi_gram_token(tokens):
    Bi=[]
    for i in range(0,len(tokens)-1):
        Bi.append(tokens[i]+tokens[i+1])
    return Bi

Below Function gives frequency dictionary of character and syllables(Sorted in descending order)

In [6]:
def freq_char_syll(Varna_viched):
  freq={}
  for shabda in Varna_viched:
      for varna in shabda:
          if varna not in freq:
              freq[varna]=1
          else:freq[varna]+=1
  freq=dict(sorted(freq.items(),key=lambda item: item[1], reverse=True))
  return freq

Below Function gives frequency dictionary of Tokens(Sorted in descending order)

In [7]:
def freq_token(Syllables):
  avritti={}
  for shabdansh in Syllables:
    if shabdansh not in avritti:
      avritti[shabdansh]=1
    else:avritti[shabdansh]+=1
  avritti=dict(sorted(avritti.items(),key=lambda item: item[1], reverse=True))
  return avritti

Below Function gives Character Separations of a list of words(tokens)

In [8]:
def Char_sep(tokens):
  Varna_viched=[]
  for word in tokens:
      temp=list()
      index=-1
      for ch in word:
          index+=1
          if ch in consonants:
              temp.append(ch+'्')
              if index+1>=len(word) or word[index+1] in consonants:
                  temp.append('अ')
          elif ch in vowels:
              temp.append(ch)
          elif ch in matras:
              temp.append(matras[ch])
      if len(temp)!=0:
          Varna_viched.append(temp)
  return Varna_viched

Below Function gives syllables of list of Words(Tokens)

In [9]:
def Char_comb(tokens):
  Syllables=[]
  for Shabda in tokens:
      Syllable=[]
      temp=str()
      index=-1
      for Varna in Shabda:
        index+=1
        if Varna in matras:
            if len(temp)==0:
              Syllable.append(matras[Varna])
              continue
            temp+=Varna
            if index+1<len(Shabda) and Shabda[index+1] in matras:
              continue
            Syllable.append(temp)
            temp=str()
        elif Varna in consonants:
            if index+1<len(Shabda) and (Shabda[index+1] in matras or Shabda[index+1]=='्'):
                temp+=Varna
                continue
            else:
                temp+=Varna
                Syllable.append(temp)
                temp=str()
        elif Varna in vowels:
            temp+=Varna
            Syllable.append(temp)
            temp=str()
        elif Varna=='्':
          if len(temp)!=0:
            temp+=Varna
      if len(Syllable)!=0:
          Syllables.append(Syllable)
  for i in range(len(Syllables)):
    for j in range(len(Syllables[i])):
        if Syllables[i][j] in matras:
          Syllables[i][j]=matras[Syllables[i][j]]
  return Syllables

# Unigram Model

In [106]:
model_prefix = "hindi_unigram"
# Train the tokenizer
spm.SentencePieceTrainer.train(input="hi_100.txt", model_prefix="hindi_unigram", vocab_size=1000, model_type="unigram")

# Load the trained tokenizer
sp = spm.SentencePieceProcessor()
sp.load(f"{model_prefix}.model")

# Tokenize text
def unigram(contents):
    tokens = sp.encode_as_pieces(contents)
    return tokens

We are now printing Top-20 Tokens ordered by their Frequency

In [107]:
tokens_unigram=unigram(contents)
for i in range(len(tokens_unigram)):
        tokens_unigram[i]=clean(tokens_unigram[i])
temp=[]
for i in tokens_unigram:
    if len(i)!=0:
        temp.append(i)
tokens_unigram=temp
freq_token_unigram=freq_token(tokens_unigram)
count=0
for key, value in freq_token_unigram.items():
    print(f"{key}: {value}",end=" ; ")
    count+=1
    if count==20:break

र: 457469 ; के: 356551 ; न: 354468 ; ल: 287236 ; म: 263003 ; ा: 251101 ; में: 240170 ; क: 226671 ; ने: 223582 ; की: 220818 ; स: 218435 ; है: 215772 ; ी: 203087 ; प: 188819 ; का: 188468 ; े: 186255 ; त: 183457 ; से: 181526 ; को: 168958 ; ज: 167971 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [108]:
freq_char_unigram=freq_char_syll(Char_sep(tokens_unigram))
count1=0
for key, value in freq_char_unigram.items():
    print(f"{key}: {value}",end=" ; ")
    count1+=1
    if count1==20:break

अ: 9292161 ; आ: 2991109 ; ए: 2318442 ; क्: 2219964 ; र्: 2115649 ; ई: 1460305 ; इ: 1432973 ; न्: 1334448 ; स्: 1283708 ; अं: 1201207 ; ह्: 1133159 ; म्: 1053237 ; त्: 980066 ; ल्: 919917 ; ओ: 896588 ; प्: 805896 ; य्: 752819 ; व्: 624743 ; द्: 607633 ; उ: 587149 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [109]:
freq_syll_unigram=freq_char_syll(Char_comb(tokens_unigram))
count2=0
for key, value in freq_syll_unigram.items():
    print(f"{key}: {value}",end=" ; ")
    count2+=1
    if count2==20:break

र: 1290207 ; क: 703020 ; न: 646780 ; इ: 615267 ; स: 596567 ; ए: 588796 ; आ: 548671 ; अं: 525803 ; प: 472955 ; ल: 456795 ; म: 439880 ; त: 395406 ; के: 395223 ; ह: 359174 ; ई: 340971 ; ने: 323639 ; ओ: 321586 ; ग: 308923 ; ब: 302876 ; उ: 295000 ; 

We are now printing Top-20 Token's Bigrams ordered by their Frequency

In [111]:
tokens_bigram=bi_gram_token(tokens_unigram)
freq_token_bigram=freq_token(tokens_bigram)
count3=0
for key, value in freq_token_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count3+=1
    if count3==20:break

केलिए: 44258 ; हैकि: 25509 ; ोंके: 21777 ; इस: 20683 ; ोंमें: 19840 ; केसाथ: 19093 ; तेहैं: 17650 ; नेके: 17618 ; ोंको: 16955 ; केबाद: 16479 ; ताहै: 16219 ; कहाकि: 16077 ; नेकहा: 16027 ; रा: 15906 ; ार: 15630 ; नके: 14843 ; ोंने: 14822 ; री: 14428 ; रु: 14420 ; ोंकी: 14122 ; 

We are now printing Top-20 Character Bigrams ordered by their Frequency

In [112]:
freq_char_bigram=bi_gram(Char_sep(tokens_unigram))
freq_char_bigram=freq_token(freq_char_bigram)
count4=0
for key, value in freq_char_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count4+=1
    if count4==20:break

र्अ: 1495965 ; क्अ: 700696 ; न्अ: 641260 ; स्अ: 625773 ; प्अ: 481293 ; ल्अ: 465885 ; म्अ: 459007 ; अर्: 456617 ; त्अ: 449518 ; क्ए: 399923 ; ह्अ: 359174 ; य्अ: 340016 ; एअं: 332062 ; अह्: 330814 ; न्ए: 323639 ; ब्अ: 305505 ; क्आ: 301641 ; ज्अ: 301116 ; ह्ऐ: 295671 ; व्अ: 288538 ; 

We are now printing Top-20 Syllable Bigrams ordered by their Frequency

In [113]:
freq_syll_bigram=bi_gram(Char_comb(tokens_unigram))
freq_syll_bigram=freq_token(freq_syll_bigram)
count5=0
for key, value in freq_syll_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count5+=1
    if count5==20:break

कर: 170718 ; ओअं: 151325 ; और: 114768 ; पर: 112276 ; इस: 86087 ; इत: 64370 ; एअं: 64089 ; एक: 56554 ; इक: 52796 ; लिए: 52487 ; नहीं: 47243 ; अंग: 43181 ; कार: 43008 ; अप: 38659 ; आई: 36797 ; किया: 36584 ; कहा: 32725 ; सम: 32620 ; यह: 30683 ; गया: 30174 ; 

# BPE Model (Vocab = 1k)

In [114]:
model_prefix = "hindi_unigram"
# Train the tokenizer
spm.SentencePieceTrainer.train(input="hi_100.txt", model_prefix="hindi_unigram", vocab_size=1000, model_type="bpe")

# Load the trained tokenizer
sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load(f"{model_prefix}.model")

# Tokenize text
def bpe_1k(contents):
    tokens_bpe = sp_bpe.encode_as_pieces(contents)
    return tokens_bpe

We are now printing Top-20 Tokens ordered by their Frequency

In [115]:
tokens_bpe1k=bpe_1k(contents)
for i in range(len(tokens_bpe1k)):
        tokens_bpe1k[i]=clean(tokens_bpe1k[i])
temp=[]
for i in tokens_bpe1k:
    if len(i)!=0:
        temp.append(i)
tokens_bpe1k=temp
freq_token_bpe1k=freq_token(tokens_bpe1k)
count6=0
for key, value in freq_token_bpe1k.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

के: 370832 ; में: 254079 ; न: 237234 ; म: 229152 ; की: 228444 ; क: 224602 ; है: 215097 ; स: 214960 ; प: 211377 ; ने: 210050 ; ल: 205377 ; त: 199691 ; ब: 184052 ; ज: 180208 ; व: 178093 ; से: 176137 ; र: 175285 ; को: 171753 ; का: 165747 ; द: 163142 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [116]:
freq_char_bpe1k=freq_char_syll(Char_sep(tokens_bpe1k))
count7=0
for key, value in freq_char_bpe1k.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 9467458 ; आ: 2991109 ; ए: 2318442 ; क्: 2219964 ; र्: 2115649 ; ई: 1460305 ; इ: 1432973 ; न्: 1334448 ; स्: 1283708 ; अं: 1201207 ; ह्: 1133159 ; म्: 1053237 ; त्: 980066 ; ल्: 919917 ; ओ: 896588 ; प्: 805896 ; य्: 752819 ; व्: 624743 ; द्: 607633 ; उ: 587149 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [117]:
freq_syll_bpe1k=freq_char_syll(Char_comb(tokens_bpe1k))
count8=0
for key, value in freq_syll_bpe1k.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

र: 1261695 ; आ: 727308 ; क: 725817 ; इ: 692621 ; न: 612164 ; स: 586132 ; ए: 551237 ; अं: 513644 ; प: 510170 ; म: 482374 ; ल: 465578 ; त: 419406 ; के: 393941 ; ह: 362088 ; ग: 334442 ; ने: 325974 ; ब: 324122 ; ओ: 313708 ; ज: 292696 ; य: 286330 ; 

We are now printing Top-20 Tokens Bigram ordered by their Frequency

In [118]:
tokens_bpe1k_bigram=bi_gram_token(tokens_bpe1k)
freq_token_bpe1k_bigram=freq_token(tokens_bpe1k_bigram)
count9=0
for key, value in freq_token_bpe1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

केलिए: 45010 ; हैकि: 25815 ; इस: 21737 ; केसाथ: 19267 ; ोंके: 18003 ; केबाद: 17287 ; नेके: 17205 ; कहाकि: 16111 ; डा: 16064 ; ोंमें: 16041 ; तेहैं: 15849 ; ोंको: 14295 ; ताहै: 13192 ; नेकहा: 12382 ; रहाहै: 12011 ; ोंकी: 11534 ; सके: 11487 ; हैऔर: 11179 ; गयाहै: 11048 ; रहेहैं: 10182 ; 

We are now printing Top-20 Characters Bigram ordered by their Frequency

In [119]:
freq_char_bpe1k_bigram=bi_gram(Char_sep(tokens_bpe1k))
freq_char_bpe1k_bigram=freq_token(freq_char_bpe1k_bigram)
count10=0
for key, value in freq_char_bpe1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

र्अ: 1451551 ; क्अ: 716847 ; स्अ: 611733 ; न्अ: 609477 ; अर्: 545362 ; प्अ: 515579 ; म्अ: 500972 ; ल्अ: 465578 ; त्अ: 451395 ; क्ए: 393941 ; ह्अ: 374275 ; आर्: 361906 ; एअं: 351250 ; य्अ: 342926 ; न्ए: 325974 ; ब्अ: 324122 ; ग्अ: 304949 ; अह्: 302089 ; व्अ: 301230 ; ज्अ: 296765 ; 

We are now printing Top-20 Syllables Bigram ordered by their Frequency

In [120]:
freq_syll_bpe1k_bigram=bi_gram(Char_comb(tokens_bpe1k))
freq_syll_bpe1k_bigram=freq_token(freq_syll_bpe1k_bigram)
count11=0
for key, value in freq_syll_bpe1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 182070 ; ओअं: 129605 ; और: 123354 ; पर: 113018 ; एअं: 79417 ; इस: 79242 ; इक: 69068 ; इत: 66500 ; कार: 60974 ; आर: 58103 ; लिए: 53882 ; आन: 52756 ; एक: 52271 ; अंग: 49942 ; आल: 48950 ; नहीं: 47219 ; अप: 46855 ; आय: 44172 ; सम: 39030 ; अंद: 36851 ; 

# BPE Model (Vocab Size=2k)

In [11]:
model_prefix = "hindi_unigram"
# Train the tokenizer
spm.SentencePieceTrainer.train(input="hi_100.txt", model_prefix="hindi_unigram", vocab_size=2000, model_type="bpe")

# Load the trained tokenizer
sp_bpe2 = spm.SentencePieceProcessor()
sp_bpe2.load(f"{model_prefix}.model")

# Tokenize text
def bpe_2k(contents):
    tokens_bpe2 = sp_bpe2.encode_as_pieces(contents)
    return tokens_bpe2

We are now printing Top-20 Tokens ordered by their Frequency

In [12]:
tokens_bpe2k=bpe_2k(contents)
for i in range(len(tokens_bpe2k)):
        tokens_bpe2k[i]=clean(tokens_bpe2k[i])
temp=[]
for i in tokens_bpe2k:
    if len(i)!=0:
        temp.append(i)
tokens_bpe2k=temp
freq_token_bpe2k=freq_token(tokens_bpe2k)
count6=0
for key, value in freq_token_bpe2k.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

के: 351330 ; में: 247089 ; की: 216147 ; है: 215097 ; ने: 179908 ; से: 166495 ; को: 165665 ; का: 150051 ; क: 140206 ; न: 127145 ; स: 120726 ; प: 119215 ; म: 119052 ; और: 114757 ; ब: 111145 ; पर: 106137 ; व: 102484 ; ल: 100856 ; कर: 95897 ; द: 95616 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [122]:
freq_char_bpe2k=freq_char_syll(Char_sep(tokens_bpe2k))
count7=0
for key, value in freq_char_bpe2k.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 8595734 ; आ: 2991109 ; ए: 2318442 ; क्: 2219964 ; र्: 2115649 ; ई: 1460305 ; इ: 1432973 ; न्: 1334448 ; स्: 1283708 ; अं: 1201207 ; ह्: 1133159 ; म्: 1053237 ; त्: 980066 ; ल्: 919917 ; ओ: 896588 ; प्: 805896 ; य्: 752819 ; व्: 624743 ; द्: 607633 ; उ: 587149 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [123]:
freq_syll_bpe2k=freq_char_syll(Char_comb(tokens_bpe2k))
count8=0
for key, value in freq_syll_bpe2k.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

र: 1175222 ; क: 675391 ; न: 585150 ; स: 568449 ; आ: 534109 ; इ: 532368 ; प: 457312 ; ए: 439851 ; ल: 429864 ; अं: 422721 ; के: 397915 ; म: 397491 ; त: 371938 ; ह: 337523 ; ने: 325974 ; का: 297194 ; ब: 295646 ; ग: 295219 ; में: 259430 ; ज: 259216 ; 

We are now printing Top-20 Tokens Bigram ordered by their Frequency

In [13]:
tokens_bpe2k_bigram=bi_gram_token(tokens_bpe2k)
freq_token_bpe2k_bigram=freq_token(tokens_bpe2k_bigram)
count9=0
for key, value in freq_token_bpe2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

केलिए: 44310 ; हैकि: 25654 ; केसाथ: 19086 ; केबाद: 16502 ; कहाकि: 16094 ; ोंके: 13984 ; ोंमें: 13283 ; नेकहा: 12098 ; नेके: 12034 ; रहाहै: 12011 ; हैऔर: 11179 ; गयाहै: 11048 ; ताहै: 10863 ; ोंको: 10425 ; रहेहैं: 10182 ; बढ: 9298 ; करनेके: 8847 ; रहीहै: 8751 ; हैइस: 8693 ; ोंकी: 8667 ; 

We are now printing Top-20 Characters Bigram ordered by their Frequency

In [14]:
freq_char_bpe2k_bigram=bi_gram(Char_sep(tokens_bpe2k))
freq_char_bpe2k_bigram=freq_token(freq_char_bpe2k_bigram)
count10=0
for key, value in freq_char_bpe2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

र्अ: 1371645 ; क्अ: 667031 ; अर्: 635184 ; न्अ: 586020 ; स्अ: 585258 ; प्अ: 465403 ; म्अ: 434732 ; ल्अ: 432167 ; त्अ: 409561 ; क्ए: 402620 ; आर्: 367651 ; एअं: 351250 ; ह्अ: 347415 ; अह्: 337138 ; न्ए: 328526 ; य्अ: 326698 ; क्आ: 304471 ; ह्ऐ: 295671 ; य्आ: 291052 ; ब्अ: 289751 ; 

We are now printing Top-20 Syllables Bigram ordered by their Frequency

In [15]:
freq_syll_bpe2k_bigram=bi_gram(Char_comb(tokens_bpe2k))
freq_syll_bpe2k_bigram=freq_token(freq_syll_bpe2k_bigram)
count11=0
for key, value in freq_syll_bpe2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 175692 ; और: 119482 ; पर: 107910 ; ओअं: 107767 ; इस: 99058 ; एअं: 58259 ; लिए: 53882 ; एक: 52271 ; कार: 52173 ; नहीं: 47219 ; इत: 46969 ; इक: 46347 ; अप: 45002 ; आर: 38983 ; किया: 36428 ; सम: 34900 ; अंग: 34829 ; आय: 32775 ; सर: 31437 ; कहा: 30937 ; 

# mBERT Model (Max_length=1k)

In [17]:
from transformers import BertTokenizer

# Load the pre-trained mBERT tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
# Tokenize the text
def mBERT1k(contents):
    text=contents
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text, max_length=1000, truncation=True)))
    return tokens

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate de

We are now printing Top-20 Tokens ordered by their Frequency

In [18]:
tokens_mBERT1k=mBERT1k(contents)
for i in range(len(tokens_mBERT1k)):
        tokens_mBERT1k[i]=clean(tokens_mBERT1k[i])
temp=[]
for i in tokens_mBERT1k:
    if len(i)!=0:
        temp.append(i)
tokens_mBERT1k=temp
freq_token_mBERT1k=freq_token(tokens_mBERT1k)
count6=0
for key, value in freq_token_mBERT1k.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

के: 29 ; में: 22 ; प: 21 ; ब: 21 ; र: 20 ; स: 20 ; आ: 18 ; न: 17 ; है: 15 ; को: 15 ; ज: 14 ; म: 14 ; ख: 13 ; ी: 13 ; ल: 13 ; द: 12 ; और: 12 ; की: 11 ; ा: 11 ; कर: 10 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [19]:
freq_char_mBERT1k=freq_char_syll(Char_sep(tokens_mBERT1k))
count7=0
for key, value in freq_char_mBERT1k.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 690 ; आ: 181 ; क्: 162 ; ए: 157 ; र्: 127 ; इ: 110 ; न्: 91 ; अं: 84 ; ई: 78 ; ह्: 77 ; स्: 77 ; ल्: 71 ; त्: 61 ; ओ: 59 ; म्: 58 ; य्: 57 ; प्: 56 ; ज्: 48 ; व्: 41 ; ऐ: 41 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [20]:
freq_syll_mBERT1k=freq_char_syll(Char_comb(tokens_mBERT1k))
count8=0
for key, value in freq_syll_mBERT1k.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

र: 92 ; क: 63 ; इ: 54 ; न: 50 ; ए: 43 ; स: 43 ; ल: 40 ; आ: 39 ; प: 39 ; ज: 34 ; अं: 33 ; के: 31 ; ब: 30 ; म: 29 ; य: 26 ; त: 24 ; या: 24 ; ओ: 24 ; ट: 24 ; व: 23 ; 

We are now printing Top-20 Tokens Bigrams ordered by their Frequency

In [21]:
tokens_mBERT1k_bigram=bi_gram_token(tokens_mBERT1k)
freq_token_mBERT1k_bigram=freq_token(tokens_mBERT1k_bigram)
count9=0
for key, value in freq_token_mBERT1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

आप: 7 ; खोज: 7 ; इस: 5 ; हैकि: 4 ; केलिए: 4 ; ोजशब्द: 4 ; ोंकी: 3 ; ्प: 3 ; कोब: 3 ; विके: 3 ; ब्रा: 3 ; ्रायन: 3 ; शब्दों: 3 ; सके: 3 ; हैइ: 2 ; आकर: 2 ; जित: 2 ; ितना: 2 ; पको: 2 ; दियाहै: 2 ; 

We are now printing Top-20 Characters Bigrams ordered by their Frequency

In [22]:
freq_char_mBERT1k_bigram=bi_gram(Char_sep(tokens_mBERT1k))
freq_char_mBERT1k_bigram=freq_token(freq_char_mBERT1k_bigram)
count10=0
for key, value in freq_char_mBERT1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

र्अ: 96 ; क्अ: 64 ; न्अ: 50 ; स्अ: 44 ; ल्अ: 40 ; प्अ: 39 ; अन्: 36 ; अर्: 36 ; ज्अ: 34 ; क्ए: 31 ; ब्अ: 30 ; म्अ: 29 ; य्अ: 28 ; एअं: 27 ; ट्अ: 26 ; त्अ: 25 ; ह्ऐ: 25 ; द्अ: 24 ; व्अ: 24 ; य्आ: 24 ; 

We are now printing Top-20 Syllables Bigrams ordered by their Frequency

In [23]:
freq_syll_mBERT1k_bigram=bi_gram(Char_comb(tokens_mBERT1k))
freq_syll_mBERT1k_bigram=freq_token(freq_syll_mBERT1k_bigram)
count11=0
for key, value in freq_syll_mBERT1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 19 ; और: 13 ; अंग: 8 ; ओज: 8 ; ओअं: 7 ; जन: 5 ; लिए: 5 ; इव: 5 ; एक: 5 ; इस: 5 ; पर: 5 ; इट: 5 ; अप: 5 ; इत: 4 ; दिया: 4 ; एअं: 4 ; इल: 4 ; इक: 4 ; अधि: 4 ; धिक: 4 ; 

# mBERT Model (Max_length=2k)

In [24]:
from transformers import BertTokenizer

# Load the pre-trained mBERT tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
# Tokenize the text
def mBERT2k(contents):
    text=contents
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text, max_length=2000, truncation=True)))
    return tokens

We are now printing Top-20 Tokens ordered by their Frequency

In [25]:
tokens_mBERT2k=mBERT2k(contents)
for i in range(len(tokens_mBERT2k)):
        tokens_mBERT2k[i]=clean(tokens_mBERT2k[i])
temp=[]
for i in tokens_mBERT2k:
    if len(i)!=0:
        temp.append(i)
tokens_mBERT2k=temp
freq_token_mBERT2k=freq_token(tokens_mBERT2k)
count6=0
for key, value in freq_token_mBERT2k.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

स: 44 ; म: 43 ; के: 41 ; र: 39 ; में: 38 ; प: 37 ; ब: 36 ; न: 30 ; की: 29 ; है: 29 ; ज: 28 ; ने: 28 ; आ: 24 ; ा: 24 ; ी: 23 ; और: 22 ; को: 21 ; व: 21 ; ु: 21 ; ल: 20 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [26]:
freq_char_mBERT2k=freq_char_syll(Char_sep(tokens_mBERT2k))
count7=0
for key, value in freq_char_mBERT2k.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 1334 ; आ: 362 ; क्: 302 ; ए: 301 ; र्: 256 ; इ: 216 ; न्: 176 ; ई: 174 ; स्: 164 ; अं: 149 ; ह्: 147 ; म्: 133 ; ल्: 125 ; त्: 116 ; ओ: 111 ; य्: 109 ; प्: 101 ; व्: 81 ; ज्: 80 ; द्: 75 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [27]:
freq_syll_mBERT2k=freq_char_syll(Char_comb(tokens_mBERT2k))
count8=0
for key, value in freq_syll_mBERT2k.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

र: 192 ; क: 111 ; इ: 105 ; स: 92 ; न: 90 ; आ: 88 ; प: 76 ; ए: 75 ; म: 67 ; ल: 63 ; अं: 61 ; उ: 58 ; ब: 53 ; ज: 52 ; य: 51 ; व: 50 ; ने: 48 ; ट: 47 ; त: 46 ; के: 45 ; 

We are now printing Top-20 Tokens Bigrams ordered by their Frequency

In [28]:
tokens_mBERT2k_bigram=bi_gram_token(tokens_mBERT2k)
freq_token_mBERT2k_bigram=freq_token(tokens_mBERT2k_bigram)
count9=0
for key, value in freq_token_mBERT2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

आप: 7 ; इस: 7 ; खोज: 7 ; केलिए: 6 ; तु: 6 ; हैकि: 5 ; जर: 5 ; मु: 5 ; स्व: 4 ; ोजशब्द: 4 ; नेकहा: 4 ; कहाकि: 4 ; मैं: 4 ; ॉय: 4 ; यले: 4 ; लेट: 4 ; ुझ: 4 ; झे: 4 ; हैइ: 3 ; नेता: 3 ; 

We are now printing Top-20 Characters Bigrams ordered by their Frequency

In [29]:
freq_char_mBERT2k_bigram=bi_gram(Char_sep(tokens_mBERT2k))
freq_char_mBERT2k_bigram=freq_token(freq_char_mBERT2k_bigram)
count10=0
for key, value in freq_char_mBERT2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

र्अ: 198 ; क्अ: 112 ; स्अ: 94 ; न्अ: 90 ; प्अ: 76 ; अर्: 68 ; म्अ: 68 ; ल्अ: 63 ; य्अ: 56 ; त्अ: 55 ; ट्अ: 55 ; ब्अ: 54 ; ज्अ: 52 ; अन्: 51 ; व्अ: 51 ; न्ए: 48 ; द्अ: 46 ; एअं: 46 ; क्ए: 45 ; ह्ऐ: 44 ; 

We are now printing Top-20 Syllables Bigrams ordered by their Frequency

In [30]:
freq_syll_mBERT2k_bigram=bi_gram(Char_comb(tokens_mBERT2k))
freq_syll_mBERT2k_bigram=freq_token(freq_syll_mBERT2k_bigram)
count11=0
for key, value in freq_syll_mBERT2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 24 ; और: 23 ; एक: 13 ; इस: 13 ; पर: 13 ; ओअं: 12 ; कार: 9 ; अंग: 9 ; ऐअं: 8 ; इट: 8 ; ओज: 8 ; कहा: 8 ; लिए: 7 ; अधि: 7 ; आय: 7 ; उक: 7 ; आर: 7 ; दिया: 6 ; एअं: 6 ; इल: 6 ; 

# IndicBERT Model( Max_length=1k)

In [31]:
from transformers import AutoTokenizer

# Replace 'ai4bharat/indic-bert' with the specific model you want to use
model_name = 'ai4bharat/indic-bert'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


def indicBERT1k(contents):
    input_text=contents
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(input_text, max_length=1000, truncation=True)))
    return tokens

We are now printing Top-20 Tokens ordered by their Frequency

In [32]:
tokens_indicBERT1k=indicBERT1k(contents)
for i in range(len(tokens_indicBERT1k)):
        tokens_indicBERT1k[i]=clean(tokens_indicBERT1k[i])
temp=[]
for i in tokens_indicBERT1k:
    if len(i)!=0:
        temp.append(i)
tokens_indicBERT1k=temp
freq_token_indicBERT1k=freq_token(tokens_indicBERT1k)
count6=0
for key, value in freq_token_indicBERT1k.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

We are now printing Top-20 Characters ordered by their Frequency

In [52]:
freq_char_indicBERT1k=freq_char_syll(Char_sep(tokens_indicBERT1k))
count7=0
for key, value in freq_char_indicBERT1k.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 169 ; क्: 18 ; न्: 16 ; र्: 15 ; ह्: 15 ; य्: 13 ; द्: 12 ; म्: 11 ; प्: 10 ; ल्: 10 ; त्: 9 ; स्: 7 ; ज्: 7 ; ब्: 5 ; आ: 4 ; व्: 4 ; भ्: 4 ; च्: 3 ; ग्: 3 ; ख्: 2 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [53]:
freq_syll_indicBERT1k=freq_char_syll(Char_comb(tokens_indicBERT1k))
count8=0
for key, value in freq_syll_indicBERT1k.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

क: 18 ; न: 16 ; र: 15 ; ह: 15 ; य: 13 ; द: 12 ; म: 11 ; प: 10 ; ल: 10 ; त: 9 ; स: 7 ; ज: 7 ; ब: 5 ; आ: 4 ; व: 4 ; भ: 4 ; च: 3 ; ग: 3 ; ख: 2 ; ए: 2 ; 

We are now printing Top-20 Tokens Bigrams ordered by their Frequency

In [54]:
tokens_indicBERT1k_bigram=bi_gram_token(tokens_indicBERT1k)
freq_token_indicBERT1k_bigram=freq_token(tokens_indicBERT1k_bigram)
count9=0
for key, value in freq_token_indicBERT1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

यह: 2 ; गय: 2 ; यथ: 2 ; महजन: 2 ; नक: 2 ; परयश: 1 ; यशह: 1 ; हय: 1 ; यद: 1 ; दई: 1 ; ईच: 1 ; चद: 1 ; ददल: 1 ; दलच: 1 ; चस: 1 ; सप: 1 ; पकरण: 1 ; करणकस: 1 ; कसह: 1 ; हनद: 1 ; 

We are now printing Top-20 Characters Bigrams ordered by their Frequency

In [55]:
freq_char_indicBERT1k_bigram=bi_gram(Char_sep(tokens_indicBERT1k))
freq_char_indicBERT1k_bigram=freq_token(freq_char_indicBERT1k_bigram)
count10=0
for key, value in freq_char_indicBERT1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

क्अ: 18 ; न्अ: 16 ; र्अ: 15 ; ह्अ: 15 ; य्अ: 13 ; अर्: 12 ; द्अ: 12 ; म्अ: 11 ; प्अ: 10 ; ल्अ: 10 ; अन्: 9 ; त्अ: 9 ; अल्: 7 ; स्अ: 7 ; ज्अ: 7 ; ब्अ: 5 ; अस्: 4 ; व्अ: 4 ; भ्अ: 4 ; अह्: 4 ; 

We are now printing Top-20 Syllables Bigrams ordered by their Frequency

In [56]:
freq_syll_indicBERT1k_bigram=bi_gram(Char_comb(tokens_indicBERT1k))
freq_syll_indicBERT1k_bigram=freq_token(freq_syll_indicBERT1k_bigram)
count11=0
for key, value in freq_syll_indicBERT1k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 6 ; तन: 3 ; पर: 2 ; दन: 2 ; हम: 2 ; मद: 2 ; हल: 2 ; लक: 2 ; मह: 2 ; हज: 2 ; यश: 1 ; दल: 1 ; रण: 1 ; कस: 1 ; नद: 1 ; आव: 1 ; रन: 1 ; आख: 1 ; तर: 1 ; जन: 1 ; 

# IndicBERT Model (Max_length=2k)

In [57]:
from transformers import AutoTokenizer

# Replace 'ai4bharat/indic-bert' with the specific model you want to use
model_name = 'ai4bharat/indic-bert'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


def indicBERT2k(contents):
    input_text=contents
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(input_text, max_length=2000, truncation=True)))
    return tokens

We are now printing Top-20 Tokens ordered by their Frequency

In [58]:
tokens_indicBERT2k=indicBERT2k(contents)
for i in range(len(tokens_indicBERT2k)):
        tokens_indicBERT2k[i]=clean(tokens_indicBERT2k[i])
temp=[]
for i in tokens_indicBERT2k:
    if len(i)!=0:
        temp.append(i)
tokens_indicBERT2k=temp
freq_token_indicBERT2k=freq_token(tokens_indicBERT2k)
count6=0
for key, value in freq_token_indicBERT2k.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

क: 9 ; ह: 8 ; य: 8 ; द: 5 ; न: 5 ; प: 4 ; म: 4 ; भ: 4 ; कर: 3 ; पर: 2 ; च: 2 ; दन: 2 ; र: 2 ; हम: 2 ; ए: 2 ; ज: 2 ; मद: 2 ; ग: 2 ; थ: 2 ; महज: 2 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [59]:
freq_char_indicBERT2k=freq_char_syll(Char_sep(tokens_indicBERT2k))
count7=0
for key, value in freq_char_indicBERT2k.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 169 ; क्: 18 ; न्: 16 ; र्: 15 ; ह्: 15 ; य्: 13 ; द्: 12 ; म्: 11 ; प्: 10 ; ल्: 10 ; त्: 9 ; स्: 7 ; ज्: 7 ; ब्: 5 ; आ: 4 ; व्: 4 ; भ्: 4 ; च्: 3 ; ग्: 3 ; ख्: 2 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [60]:
freq_syll_indicBERT2k=freq_char_syll(Char_comb(tokens_indicBERT2k))
count8=0
for key, value in freq_syll_indicBERT2k.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

क: 18 ; न: 16 ; र: 15 ; ह: 15 ; य: 13 ; द: 12 ; म: 11 ; प: 10 ; ल: 10 ; त: 9 ; स: 7 ; ज: 7 ; ब: 5 ; आ: 4 ; व: 4 ; भ: 4 ; च: 3 ; ग: 3 ; ख: 2 ; ए: 2 ; 

We are now printing Top-20 Tokens Bigrams ordered by their Frequency

In [61]:
tokens_indicBERT2k_bigram=bi_gram_token(tokens_indicBERT2k)
freq_token_indicBERT2k_bigram=freq_token(tokens_indicBERT2k_bigram)
count9=0
for key, value in freq_token_indicBERT2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

यह: 2 ; गय: 2 ; यथ: 2 ; महजन: 2 ; नक: 2 ; परयश: 1 ; यशह: 1 ; हय: 1 ; यद: 1 ; दई: 1 ; ईच: 1 ; चद: 1 ; ददल: 1 ; दलच: 1 ; चस: 1 ; सप: 1 ; पकरण: 1 ; करणकस: 1 ; कसह: 1 ; हनद: 1 ; 

We are now printing Top-20 Character Bigrams ordered by their Frequency

In [62]:
freq_char_indicBERT2k_bigram=bi_gram(Char_sep(tokens_indicBERT2k))
freq_char_indicBERT2k_bigram=freq_token(freq_char_indicBERT2k_bigram)
count10=0
for key, value in freq_char_indicBERT2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

क्अ: 18 ; न्अ: 16 ; र्अ: 15 ; ह्अ: 15 ; य्अ: 13 ; अर्: 12 ; द्अ: 12 ; म्अ: 11 ; प्अ: 10 ; ल्अ: 10 ; अन्: 9 ; त्अ: 9 ; अल्: 7 ; स्अ: 7 ; ज्अ: 7 ; ब्अ: 5 ; अस्: 4 ; व्अ: 4 ; भ्अ: 4 ; अह्: 4 ; 

We are now printing Top-20 Syllables Bigrams ordered by their Frequency

In [63]:
freq_syll_indicBERT2k_bigram=bi_gram(Char_comb(tokens_indicBERT2k))
freq_syll_indicBERT2k_bigram=freq_token(freq_syll_indicBERT2k_bigram)
count11=0
for key, value in freq_syll_indicBERT2k_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 6 ; तन: 3 ; पर: 2 ; दन: 2 ; हम: 2 ; मद: 2 ; हल: 2 ; लक: 2 ; मह: 2 ; हज: 2 ; यश: 1 ; दल: 1 ; रण: 1 ; कस: 1 ; नद: 1 ; आव: 1 ; रन: 1 ; आख: 1 ; तर: 1 ; जन: 1 ; 

# WhiteSpaceTokenizer

In [64]:
def WhiteSpaceTokenizer(contents):
    return contents.split()

We are now printing Top-20 Tokens ordered by their Frequency

In [65]:
tokens_WhiteSpaceTokenizer=WhiteSpaceTokenizer(contents)
temp=[]
for i in tokens_WhiteSpaceTokenizer:
    if len(i)!=0:
        temp.append(i)
tokens_WhiteSpaceTokenizer=temp
freq_token_WhiteSpaceTokenizer=freq_token(tokens_WhiteSpaceTokenizer)
count6=0
for key, value in freq_token_WhiteSpaceTokenizer.items():
    print(f"{key}: {value}",end=" ; ")
    count6+=1
    if count6==20:break

में: 3 ; है: 3 ; की: 2 ; भी: 2 ; के: 2 ; को: 2 ; गया: 2 ; था: 2 ; महाजन: 2 ; प्रियांशु: 1 ; ह्युंदई: 1 ; चाँद: 1 ; दिलचस्प: 1 ; कर्ण: 1 ; कंस: 1 ; हिन्दी: 1 ; आवेदन: 1 ; करने: 1 ; आखिरी: 1 ; तारीख: 1 ; 

We are now printing Top-20 Characters ordered by their Frequency

In [66]:
freq_char_WhiteSpaceTokenizer=freq_char_syll(Char_sep(tokens_WhiteSpaceTokenizer))
count7=0
for key, value in freq_char_WhiteSpaceTokenizer.items():
    print(f"{key}: {value}",end=" ; ")
    count7+=1
    if count7==20:break

अ: 66 ; आ: 38 ; क्: 18 ; न्: 16 ; ए: 16 ; र्: 15 ; इ: 15 ; ह्: 15 ; ई: 15 ; य्: 13 ; द्: 12 ; म्: 11 ; प्: 10 ; अं: 10 ; ल्: 10 ; त्: 9 ; स्: 7 ; ज्: 7 ; ओ: 7 ; उ: 5 ; 

We are now printing Top-20 Syllables ordered by their Frequency

In [67]:
freq_syll_WhiteSpaceTokenizer=freq_char_syll(Char_comb(tokens_WhiteSpaceTokenizer))
count8=0
for key, value in freq_syll_WhiteSpaceTokenizer.items():
    print(f"{key}: {value}",end=" ; ")
    count8+=1
    if count8==20:break

न: 7 ; या: 7 ; स: 6 ; र: 6 ; त: 6 ; क: 5 ; द: 4 ; ल: 4 ; में: 4 ; आ: 4 ; ज: 4 ; ह: 4 ; दि: 3 ; ने: 3 ; री: 3 ; है: 3 ; ना: 3 ; को: 3 ; ब: 3 ; का: 3 ; 

We are now printing Top-20 Tokens Bigrams ordered by their Frequency

In [68]:
tokens_WhiteSpaceTokenizer_bigram=bi_gram_token(tokens_WhiteSpaceTokenizer)
freq_token_WhiteSpaceTokenizer_bigram=freq_token(tokens_WhiteSpaceTokenizer_bigram)
count9=0
for key, value in freq_token_WhiteSpaceTokenizer_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count9+=1
    if count9==20:break

प्रियांशुह्युंदई: 1 ; ह्युंदईचाँद: 1 ; चाँददिलचस्प: 1 ; दिलचस्पकर्ण: 1 ; कर्णकंस: 1 ; कंसहिन्दी: 1 ; हिन्दीमें: 1 ; मेंआवेदन: 1 ; आवेदनकरने: 1 ; करनेकी: 1 ; कीआखिरी: 1 ; आखिरीतारीख: 1 ; तारीखजनवरी: 1 ; जनवरीहै: 1 ; हैइतनी: 1 ; इतनीदुआ: 1 ; दुआकर: 1 ; करदो: 1 ; दोहमारे: 1 ; हमारेलिए: 1 ; 

We are now printing Top-20 Character Bigrams ordered by their Frequency

In [69]:
freq_char_WhiteSpaceTokenizer_bigram=bi_gram(Char_sep(tokens_WhiteSpaceTokenizer))
freq_char_WhiteSpaceTokenizer_bigram=freq_token(freq_char_WhiteSpaceTokenizer_bigram)
count10=0
for key, value in freq_char_WhiteSpaceTokenizer_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count10+=1
    if count10==20:break

य्आ: 9 ; अन्: 9 ; न्अ: 7 ; र्अ: 7 ; अर्: 6 ; स्अ: 6 ; त्अ: 6 ; क्अ: 5 ; आर्: 5 ; अह्: 5 ; ल्अ: 4 ; म्ए: 4 ; एअं: 4 ; ज्अ: 4 ; ह्अ: 4 ; अम्: 4 ; आज्: 4 ; इय्: 3 ; द्ई: 3 ; द्अ: 3 ; 

We are now printing Top-20 Syllables Bigrams ordered by their Frequency

In [70]:
freq_syll_WhiteSpaceTokenizer_bigram=bi_gram(Char_comb(tokens_WhiteSpaceTokenizer))
freq_syll_WhiteSpaceTokenizer_bigram=freq_token(freq_syll_WhiteSpaceTokenizer_bigram)
count11=0
for key, value in freq_syll_WhiteSpaceTokenizer_bigram.items():
    print(f"{key}: {value}",end=" ; ")
    count11+=1
    if count11==20:break

कर: 3 ; जन: 3 ; तना: 2 ; गया: 2 ; महा: 2 ; हाज: 2 ; प्रियां: 1 ; यांशु: 1 ; ह्युंद: 1 ; दई: 1 ; चाँद: 1 ; दिल: 1 ; लच: 1 ; चस्प: 1 ; कर्ण: 1 ; कंस: 1 ; हिन्दी: 1 ; आवे: 1 ; वेद: 1 ; दन: 1 ; 

# Q3 Loading Sentences

In [71]:
file1=open('cs689_assignment.txt','r')
Sentences=file1.read()
file1.close()
Sentences=Sentences.split("\n")

Unigrams Tokens

In [72]:
Model_Tokens=[]
for sentence in Sentences:
    Model_Tokens.append(unigram(sentence))
for token in Model_Tokens:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens:
    if len(i)!=0:
        temp.append(i)
Model_Tokens=temp
print(Model_Tokens)

[['आपको', 'प', 'ता', 'होगा', 'कि', 'मो', 'टा', 'पा', 'भी', 'हा', 'र्ट', 'अ', 'ट', 'ै', 'क', 'का', 'एक', 'प्रमुख', 'कारण', 'है', ''], ['कोई', 'भी', 'ब', 'डा', 'ने', 'ता', 'उनके', 'क्षेत्र', 'में', 'प्र', 'चार', 'के', 'लिए', 'नहीं', 'गया', ''], ['थी', 'ं', 'डि', 'प्र', 'ेशन', 'की', 'शि', 'कार'], ['ऐसे', 'में', 'स्व', 'प', '्', 'ना', 'के', 'सामने', 'खुद', 'को', 'सा', 'ब', 'ित', 'करने', 'और', 'उन', 'लोगों', 'को', 'ग', 'ल', 'त', 'सा', 'ब', 'ित', 'करने', 'की', 'च', 'ु', 'न', 'ौ', 'ती', 'थी', '', 'जो', 'उनके', 'काफी', 'करीब', 'रह', 'ते', 'हुए', 'भी', 'उन्हें', 'पद', 'क', 'का', 'दा', 'वे', 'दार', 'नहीं', 'मान', 'रहे', 'थे', ''], ['देश', 'भ', 'र', 'में', 'महिला', 'सुरक्षा', 'को', 'लेकर', 'ह', 'ंग', 'ा', 'मा', 'म', 'चा', 'है', 'और', 'स', 'ख', '्', 'त', 'कानून', 'की', 'मांग', 'की', 'जा', 'रही', 'है', ''], ['हाल', 'ही', 'में', 'दी', 'प', 'िक', 'ा', 'पा', 'द', 'ु', 'को', 'ण', 'की', 'फिल्म', 'पद', '्', 'मा', 'व', 'ती', 'को', 'लेकर', 'काफी', 'विरोध', 'चल', 'रहा', 'है', '', 'और', 'कई', 'राज्य', 'ों', 

# Loading Solution

In [73]:
file2=open('cs689_assignment_sol.txt','r')
Solns=file2.read()
file2.close()
Solns=Solns.split("\n")
Soln_Tokens=[]
for soln in Solns:
    Soln_Tokens.append(soln.split(","))
print(Soln_Tokens)

[['आपको पता होगा', ' कि मोटापा भी', ' हार्ट अटैक का एक प्रमुख कारण है।'], ['कोई भी', ' बड़ा नेता', ' उनके क्षेत्र में', ' प्रचार के लिए', ' नहीं गया।'], ['थीं', ' डिप्रेशन की', ' शिकार'], ['ऐसे में', ' स्वप्ना के सामने', ' खुद को साबित करने और', ' उन लोगों को', ' गलत साबित करने की चुनौती थी', ' जो उनके काफी करीब रहते हुए', ' भी उन्हें पदक का दावेदार नहीं मान रहे थे। '], ['देशभर में', ' महिला सुरक्षा को लेकर', ' हंगामा मचा है', ' और', ' सख्त कानून की मांग की जा रही है.'], ['हाल ही में', 'दीपिका पादुकोण की फिल्म पद्मावती को लेकर', ' काफी विरोध चल रहा है.', ' और', ' कई राज्यों में', ' तो इस फिल्म को बैन भी कर दिया गया है.', ' वहीँ भंसाली और दीपिका को धमकियाँ भी दी जा रही हैं.'], ['खासकर', ' फेसबुक पर', ' हमारे हिन्दीभाषी राज्यों के', ' जीवन की विविधता और', ' दैनंदिन जीवन की चुनौतियां', ' नजर नहीं आ रही हैं।'], ['इसके बादवाले', ' एक माह तक', ' कुछ पका हुआ आहार', ' दिया जा सकता है। '], ['फिलहाल', ' जीआरपी पुलिस ने', ' आरोपी के खिलाफ', ' धारा 376 और पॉक्सो एक्ट के तहत', ' मामला दर्ज कर लिया ह

BPE(V=1k) Tokens

In [74]:
Model_Tokens_BPE_1k=[]
for sentence in Sentences:
    Model_Tokens_BPE_1k.append(bpe_1k(sentence))
for token in Model_Tokens_BPE_1k:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens_BPE_1k:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_BPE_1k=temp
print(Model_Tokens_BPE_1k)

[['आपको', 'प', 'ता', 'होगा', 'कि', 'मो', 'टा', 'पा', 'भी', 'ह', 'ार्ट', 'अ', 'ट', 'ै', 'क', 'का', 'एक', 'प्र', 'मु', 'ख', 'कारण', 'है', ''], ['कोई', 'भी', 'ब', 'डा', 'नेता', 'उनके', 'क्षेत्र', 'में', 'प्र', 'चार', 'के', 'लिए', 'नहीं', 'गया', ''], ['थी', 'ं', 'डि', 'प्र', 'ेशन', 'की', 'शिक', 'ार'], ['ऐसे', 'में', 'स्व', 'प्', 'ना', 'के', 'सामने', 'खु', 'द', 'को', 'सा', 'ब', 'ित', 'करने', 'और', 'उन', 'लोगों', 'को', 'ग', 'ल', 'त', 'सा', 'ब', 'ित', 'करने', 'की', 'चु', 'न', 'ौ', 'ती', 'थी', '', 'जो', 'उनके', 'काफी', 'करीब', 'रह', 'ते', 'हुए', 'भी', 'उन्हें', 'पद', 'क', 'का', 'दा', 'वे', 'दार', 'नहीं', 'मान', 'रहे', 'थे', ''], ['देश', 'भ', 'र', 'में', 'महिला', 'सुर', 'क्षा', 'को', 'लेकर', 'ह', 'ंग', 'ाम', 'ा', 'म', 'चा', 'है', 'और', 'स', 'ख', '्त', 'क', 'ान', 'ून', 'की', 'मांग', 'की', 'जा', 'रही', 'है', ''], ['हाल', 'ही', 'में', 'दी', 'प', 'िका', 'पा', 'द', 'ु', 'को', 'ण', 'की', 'फिल्म', 'पद', '्म', 'ा', 'व', 'ती', 'को', 'लेकर', 'काफी', 'वि', 'रो', 'ध', 'चल', 'रहा', 'है', '', 'और', 'कई', 'रा

BPE(V=2k) Tokens

In [75]:
Model_Tokens_BPE_2k=[]
for sentence in Sentences:
    Model_Tokens_BPE_2k.append(bpe_2k(sentence))
for token in Model_Tokens_BPE_2k:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens_BPE_2k:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_BPE_2k=temp
print(Model_Tokens_BPE_2k)

[['आपको', 'पता', 'होगा', 'कि', 'मो', 'टा', 'पा', 'भी', 'ह', 'ार्ट', 'अ', 'ट', 'ै', 'क', 'का', 'एक', 'प्रमुख', 'कारण', 'है', ''], ['कोई', 'भी', 'बडा', 'नेता', 'उनके', 'क्षेत्र', 'में', 'प्र', 'चार', 'के', 'लिए', 'नहीं', 'गया', ''], ['थीं', 'डि', 'प्र', 'ेशन', 'की', 'शिक', 'ार'], ['ऐसे', 'में', 'स्व', 'प्', 'ना', 'के', 'सामने', 'खुद', 'को', 'सा', 'ब', 'ित', 'करने', 'और', 'उन', 'लोगों', 'को', 'गल', 'त', 'सा', 'ब', 'ित', 'करने', 'की', 'चुन', 'ौ', 'ती', 'थी', '', 'जो', 'उनके', 'काफी', 'करीब', 'रहते', 'हुए', 'भी', 'उन्हें', 'पद', 'क', 'का', 'दा', 'वे', 'दार', 'नहीं', 'मान', 'रहे', 'थे', ''], ['देश', 'भर', 'में', 'महिला', 'सुरक्षा', 'को', 'लेकर', 'ह', 'ंग', 'ामा', 'म', 'चा', 'है', 'और', 'स', 'ख', '्त', 'कानून', 'की', 'मांग', 'की', 'जा', 'रही', 'है', ''], ['हाल', 'ही', 'में', 'दी', 'प', 'िका', 'पा', 'द', 'ु', 'को', 'ण', 'की', 'फिल्म', 'पद', '्म', 'ा', 'व', 'ती', 'को', 'लेकर', 'काफी', 'विरोध', 'चल', 'रहा', 'है', '', 'और', 'कई', 'राज', '्यों', 'में', 'तो', 'इस', 'फिल्म', 'को', 'बै', 'न', 'भी', '

mBERT(Max_length=1k) Tokens

In [76]:
Model_Tokens_Bert1k=[]
for sentence in Sentences:
    Model_Tokens_Bert1k.append(mBERT1k(sentence))
for token in Model_Tokens_Bert1k:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens_Bert1k:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_Bert1k=temp
print(Model_Tokens_Bert1k)

[['', 'आप', 'क', 'पत', 'ह', 'ग', 'क', 'मट', 'प', 'भ', 'हर', 'ट', 'अटक', 'क', 'एक', 'पर', 'मख', 'करण', 'ह', '', ''], ['', 'कई', 'भ', 'बड', 'नत', 'उन', 'क', 'क', 'ष', 'तर', 'म', 'पर', 'चर', 'क', 'ल', 'ए', 'न', 'ह', 'ग', 'य', '', ''], ['', 'थ', 'ड', 'पर', 'शन', 'क', 'श', 'कर', ''], ['', 'ऐस', 'म', 'सव', 'पन', 'क', 'समन', 'खद', 'क', 'सब', 'त', 'करन', 'और', 'उन', 'लग', 'क', 'गलत', 'सब', 'त', 'करन', 'क', '', 'चन', 'त', 'थ', '', 'ज', 'उन', 'क', 'कफ', 'कर', 'ब', 'रह', 'त', 'ह', 'ए', 'भ', 'उन', 'ह', 'पदक', 'क', '', 'दव', 'दर', 'न', 'ह', 'मन', 'रह', 'थ', '', '', ''], ['', 'दश', 'भर', 'म', 'महल', 'सरक', 'ष', 'क', '', 'लकर', 'ह', 'गम', 'मच', 'ह', 'और', 'सख', 'त', 'कन', 'न', 'क', 'मग', 'क', 'ज', 'रह', 'ह', '', ''], ['', 'हल', 'ह', 'म', 'द', 'पक', 'पदक', 'ण', 'क', 'फल', 'म', 'पद', 'म', 'वत', 'क', '', 'लकर', 'कफ', 'वर', 'ध', 'चल', 'रह', 'ह', '', 'और', 'कई', 'रज', 'य', 'म', 'त', 'इस', 'फल', 'म', 'क', 'बन', 'भ', 'कर', 'द', 'य', 'ग', 'य', 'ह', '', 'वह', 'भ', 'सल', 'और', 'द', 'पक', 'क', 'धमक', 'य', 'भ', 

mBERT(Max_length=2k) Tokens

In [78]:
Model_Tokens_Bert2k=[]
for sentence in Sentences:
    Model_Tokens_Bert2k.append(mBERT2k(sentence))
for token in Model_Tokens_Bert2k:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens_Bert2k:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_Bert2k=temp
print(Model_Tokens_Bert2k)

[['', 'आप', 'क', 'पत', 'ह', 'ग', 'क', 'मट', 'प', 'भ', 'हर', 'ट', 'अटक', 'क', 'एक', 'पर', 'मख', 'करण', 'ह', '', ''], ['', 'कई', 'भ', 'बड', 'नत', 'उन', 'क', 'क', 'ष', 'तर', 'म', 'पर', 'चर', 'क', 'ल', 'ए', 'न', 'ह', 'ग', 'य', '', ''], ['', 'थ', 'ड', 'पर', 'शन', 'क', 'श', 'कर', ''], ['', 'ऐस', 'म', 'सव', 'पन', 'क', 'समन', 'खद', 'क', 'सब', 'त', 'करन', 'और', 'उन', 'लग', 'क', 'गलत', 'सब', 'त', 'करन', 'क', '', 'चन', 'त', 'थ', '', 'ज', 'उन', 'क', 'कफ', 'कर', 'ब', 'रह', 'त', 'ह', 'ए', 'भ', 'उन', 'ह', 'पदक', 'क', '', 'दव', 'दर', 'न', 'ह', 'मन', 'रह', 'थ', '', '', ''], ['', 'दश', 'भर', 'म', 'महल', 'सरक', 'ष', 'क', '', 'लकर', 'ह', 'गम', 'मच', 'ह', 'और', 'सख', 'त', 'कन', 'न', 'क', 'मग', 'क', 'ज', 'रह', 'ह', '', ''], ['', 'हल', 'ह', 'म', 'द', 'पक', 'पदक', 'ण', 'क', 'फल', 'म', 'पद', 'म', 'वत', 'क', '', 'लकर', 'कफ', 'वर', 'ध', 'चल', 'रह', 'ह', '', 'और', 'कई', 'रज', 'य', 'म', 'त', 'इस', 'फल', 'म', 'क', 'बन', 'भ', 'कर', 'द', 'य', 'ग', 'य', 'ह', '', 'वह', 'भ', 'सल', 'और', 'द', 'पक', 'क', 'धमक', 'य', 'भ', 

IndicBERT(Max_length=1k) Tokens

In [77]:
Model_Tokens_IndicBert1k=[]
for sentence in Sentences:
    Model_Tokens_IndicBert1k.append(indicBERT1k(sentence))
for token in Model_Tokens_IndicBert1k:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens_IndicBert1k:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_IndicBert1k=temp
print(Model_Tokens_IndicBert1k)

[['', 'आप', 'क', 'पत', 'ह', 'ग', 'क', 'मट', 'प', 'भ', 'हर', 'ट', 'अटक', 'क', 'एक', 'पर', 'मख', 'करण', 'ह', '', ''], ['', 'कई', 'भ', 'बड', 'नत', 'उन', 'क', 'क', 'ष', 'तर', 'म', 'पर', 'चर', 'क', 'ल', 'ए', 'न', 'ह', 'ग', 'य', '', ''], ['', 'थ', 'ड', 'पर', 'शन', 'क', 'श', 'कर', ''], ['', 'ऐस', 'म', 'सव', 'पन', 'क', 'समन', 'खद', 'क', 'सब', 'त', 'करन', 'और', 'उन', 'लग', 'क', 'गलत', 'सब', 'त', 'करन', 'क', '', 'चन', 'त', 'थ', '', 'ज', 'उन', 'क', 'कफ', 'कर', 'ब', 'रह', 'त', 'ह', 'ए', 'भ', 'उन', 'ह', 'पदक', 'क', '', 'दव', 'दर', 'न', 'ह', 'मन', 'रह', 'थ', '', '', ''], ['', 'दश', 'भर', 'म', 'महल', 'सरक', 'ष', 'क', '', 'लकर', 'ह', 'गम', 'मच', 'ह', 'और', 'सख', 'त', 'कन', 'न', 'क', 'मग', 'क', 'ज', 'रह', 'ह', '', ''], ['', 'हल', 'ह', 'म', 'द', 'पक', 'पदक', 'ण', 'क', 'फल', 'म', 'पद', 'म', 'वत', 'क', '', 'लकर', 'कफ', 'वर', 'ध', 'चल', 'रह', 'ह', '', 'और', 'कई', 'रज', 'य', 'म', 'त', 'इस', 'फल', 'म', 'क', 'बन', 'भ', 'कर', 'द', 'य', 'ग', 'य', 'ह', '', 'वह', 'भ', 'सल', 'और', 'द', 'पक', 'क', 'धमक', 'य', 'भ', 

IndicBERT(Max_length=2k) Tokens

In [79]:
Model_Tokens_IndicBert2k=[]
for sentence in Sentences:
    Model_Tokens_IndicBert2k.append(indicBERT2k(sentence))
for token in Model_Tokens_IndicBert2k:
    for i in range(len(token)):
        token[i]=clean(token[i])
temp=[]
for i in Model_Tokens_IndicBert2k:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_IndicBert2k=temp
print(Model_Tokens_IndicBert2k)

[['', 'आप', 'क', 'पत', 'ह', 'ग', 'क', 'मट', 'प', 'भ', 'हर', 'ट', 'अटक', 'क', 'एक', 'पर', 'मख', 'करण', 'ह', '', ''], ['', 'कई', 'भ', 'बड', 'नत', 'उन', 'क', 'क', 'ष', 'तर', 'म', 'पर', 'चर', 'क', 'ल', 'ए', 'न', 'ह', 'ग', 'य', '', ''], ['', 'थ', 'ड', 'पर', 'शन', 'क', 'श', 'कर', ''], ['', 'ऐस', 'म', 'सव', 'पन', 'क', 'समन', 'खद', 'क', 'सब', 'त', 'करन', 'और', 'उन', 'लग', 'क', 'गलत', 'सब', 'त', 'करन', 'क', '', 'चन', 'त', 'थ', '', 'ज', 'उन', 'क', 'कफ', 'कर', 'ब', 'रह', 'त', 'ह', 'ए', 'भ', 'उन', 'ह', 'पदक', 'क', '', 'दव', 'दर', 'न', 'ह', 'मन', 'रह', 'थ', '', '', ''], ['', 'दश', 'भर', 'म', 'महल', 'सरक', 'ष', 'क', '', 'लकर', 'ह', 'गम', 'मच', 'ह', 'और', 'सख', 'त', 'कन', 'न', 'क', 'मग', 'क', 'ज', 'रह', 'ह', '', ''], ['', 'हल', 'ह', 'म', 'द', 'पक', 'पदक', 'ण', 'क', 'फल', 'म', 'पद', 'म', 'वत', 'क', '', 'लकर', 'कफ', 'वर', 'ध', 'चल', 'रह', 'ह', '', 'और', 'कई', 'रज', 'य', 'म', 'त', 'इस', 'फल', 'म', 'क', 'बन', 'भ', 'कर', 'द', 'य', 'ग', 'य', 'ह', '', 'वह', 'भ', 'सल', 'और', 'द', 'पक', 'क', 'धमक', 'य', 'भ', 

WhiteSpaceTokenizer Tokens

In [92]:
Model_Tokens_WhiteSpaceTokenizer=[]
for sentence in Sentences:
    Model_Tokens_WhiteSpaceTokenizer.append(WhiteSpaceTokenizer(sentence))
temp=[]
for i in Model_Tokens_WhiteSpaceTokenizer:
    if len(i)!=0:
        temp.append(i)
Model_Tokens_WhiteSpaceTokenizer=temp
print(Model_Tokens_WhiteSpaceTokenizer)

[['आपको', 'पता', 'होगा', 'कि', 'मोटापा', 'भी', 'हार्ट', 'अटैक', 'का', 'एक', 'प्रमुख', 'कारण', 'है।'], ['कोई', 'भी', 'बड़ा', 'नेता', 'उनके', 'क्षेत्र', 'में', 'प्रचार', 'के', 'लिए', 'नहीं', 'गया।'], ['थीं', 'डिप्रेशन', 'की', 'शिकार'], ['ऐसे', 'में', 'स्वप्ना', 'के', 'सामने', 'खुद', 'को', 'साबित', 'करने', 'और', 'उन', 'लोगों', 'को', 'गलत', 'साबित', 'करने', 'की', 'चुनौती', 'थी,', 'जो', 'उनके', 'काफी', 'करीब', 'रहते', 'हुए', 'भी', 'उन्हें', 'पदक', 'का', 'दावेदार', 'नहीं', 'मान', 'रहे', 'थे।'], ['देशभर', 'में', 'महिला', 'सुरक्षा', 'को', 'लेकर', 'हंगामा', 'मचा', 'है', 'और', 'सख्त', 'कानून', 'की', 'मांग', 'की', 'जा', 'रही', 'है.'], ['हाल', 'ही', 'में', 'दीपिका', 'पादुकोण', 'की', 'फिल्म', 'पद्मावती', 'को', 'लेकर', 'काफी', 'विरोध', 'चल', 'रहा', 'है.', 'और', 'कई', 'राज्यों', 'में', 'तो', 'इस', 'फिल्म', 'को', 'बैन', 'भी', 'कर', 'दिया', 'गया', 'है.', 'वहीँ', 'भंसाली', 'और', 'दीपिका', 'को', 'धमकियाँ', 'भी', 'दी', 'जा', 'रही', 'हैं.'], ['खासकर', 'फेसबुक', 'पर', 'हमारे', 'हिन्दीभाषी', 'राज्यों', 'के', 

Function to calculate Precision,Recall,F-Score

In [94]:
def calculate_metrics(ground_truth, tokenizer_output):
    t_p=0.0
    f_p=0.0
    f_n=0.0
    for i in range (0, len(ground_truth)):
        t_p+=len(set(ground_truth[i]) & set(tokenizer_output[i]))
        f_p+=len(set(tokenizer_output[i]) - set(ground_truth[i]))
        f_n+=len(set(ground_truth[i]) - set(tokenizer_output[i]))
    if t_p + f_p != 0:
        precision = t_p / (t_p + f_p) 
    else: precision = 0
    if t_p + f_n != 0:
        recall = t_p / (t_p + f_n)  
    else:recall =  0
    if precision + recall != 0:
        f_score = 2 * (precision * recall) / (precision + recall)  
    else:f_score =  0
    return precision, recall, f_score

# Precision,Recall,F-Scores of Various Models 

In [81]:
p, r, f = calculate_metrics(Soln_Tokens,Model_Tokens)
print("Unigram Metrics:")
print(f"Precision: {p}")
print(f"Recall: {r}")
print(f"F-score: {f}")

Unigram Metrics:
Precision: 0.002366863905325444
Recall: 0.010810810810810811
F-score: 0.0038834951456310682


In [82]:
p_bpe1k, r_bpe1k, f_bpe1k = calculate_metrics(Soln_Tokens,Model_Tokens_BPE_1k)
print("BPE1k Metrics:")
print(f"Precision: {p_bpe1k}")
print(f"Recall: {r_bpe1k}")
print(f"F-score: {f_bpe1k}")

BPE1k Metrics:
Precision: 0.00228310502283105
Recall: 0.010810810810810811
F-score: 0.003770028275212064


In [84]:
p_bpe2k, r_bpe2k, f_bpe2k = calculate_metrics(Soln_Tokens,Model_Tokens_BPE_2k)
print("BPE2k Metrics:")
print(f"Precision: {p_bpe2k}")
print(f"Recall: {r_bpe2k}")
print(f"F-score: {f_bpe2k}")

BPE2k Metrics:
Precision: 0.0038022813688212928
Recall: 0.016216216216216217
F-score: 0.006160164271047227


In [87]:
p_bert1k, r_bert1k, f_bert1k = calculate_metrics(Soln_Tokens,Model_Tokens_Bert1k)
print("mBERT1k Metrics:")
print(f"Precision: {p_bert1k}")
print(f"Recall: {r_bert1k}")
print(f"F-score: {f_bert1k}")

mBERT1k Metrics:
Precision: 0.001594896331738437
Recall: 0.005405405405405406
F-score: 0.0024630541871921183


In [88]:
p_bert2k, r_bert2k, f_bert2k = calculate_metrics(Soln_Tokens,Model_Tokens_Bert2k)
print("mBERT2k Metrics:")
print(f"Precision: {p_bert2k}")
print(f"Recall: {r_bert2k}")
print(f"F-score: {f_bert2k}")

mBERT2k Metrics:
Precision: 0.001594896331738437
Recall: 0.005405405405405406
F-score: 0.0024630541871921183


In [90]:
p_ibert1k, r_ibert1k, f_ibert1k = calculate_metrics(Soln_Tokens,Model_Tokens_IndicBert1k)
print("IndicBERT1k Metrics:")
print(f"Precision: {p_ibert1k}")
print(f"Recall: {r_ibert1k}")
print(f"F-score: {f_ibert1k}")

IndicBERT1k Metrics:
Precision: 0.001594896331738437
Recall: 0.005405405405405406
F-score: 0.0024630541871921183


In [91]:
p_ibert2k, r_ibert2k, f_ibert2k = calculate_metrics(Soln_Tokens,Model_Tokens_IndicBert2k)
print("IndicBERT2k Metrics:")
print(f"Precision: {p_ibert2k}")
print(f"Recall: {r_ibert2k}")
print(f"F-score: {f_ibert2k}")

IndicBERT2k Metrics:
Precision: 0.001594896331738437
Recall: 0.005405405405405406
F-score: 0.0024630541871921183


In [93]:
p_WhiteSpaceTokenizer, r_WhiteSpaceTokenizer, f_WhiteSpaceTokenizer = calculate_metrics(Soln_Tokens,Model_Tokens_WhiteSpaceTokenizer)
print("WhiteSpaceTokenizer Metrics:")
print(f"Precision: {p_WhiteSpaceTokenizer}")
print(f"Recall: {r_WhiteSpaceTokenizer}")
print(f"F-score: {f_WhiteSpaceTokenizer}")

WhiteSpaceTokenizer Metrics:
Precision: 0.009541984732824428
Recall: 0.02702702702702703
F-score: 0.014104372355430184
